Bert(Bidirectional Encoder Representation from Transformers) 是个只有**双向编码器**的 Transformer，虽然结构很简单，但效果很好
分为两个版本：
- Base: #blocks 12, hidden size 768,    #heads 12,  #parameters 110M
- Large:#blocks 24, hidden size 1024,   #heads 16,  #parameters 340M

创新点：
- 输入
- Loss 函数

<cls> 表示句子的分类，<sep> 表示两句话之间的断句符号

### 训练任务1：带掩码的语言模型
Bert 之所以是双向的，是因为它不需要预测未来，只做完形填空。
带掩码的语言模型每次随机（15%概率)将一些词元换成<mask>
因为微调任务中不出现<mask>
- 80%概率下，将选中的词元变成<mask>
- 10%概率下换成一个随机词元
- 10%概率下保持原有的词元

目的是不要看到 mask 就预测

### 训练任务2：下一句子预测
- 预测一个句子对中两个句子是不是相邻
>每次给出一对句子，预测句子在原始数据中是不是相邻的。

- 因此，在训练样本中：
    - 50% 概率选择**相邻**句子对：<cls>this movie is great<Sep>i like it <sep>
    - 50% 概率选择**随机**句子对：<cls>this movie is great<Sep>hello world<sep>

### 总结
- BERT针对微调设计
- 基于Transformer的编码器做了如下修改
    - 模型更大，训川练数据更多
    - 输入句子对，片段嵌入，可学习的位置编码
- 训练时使用两个任务：
    - 带掩码的语言模型
    - 下一个句子预测

## Bert 模型

In [1]:
import torch
from torch import nn
from d2l import torch as d2l

### 1. 输入表示
NLP 有些任务（如情感分析）以单个文本作为输入，而有些任务（如自然语言推断）以一对文本序列作为输入。bert 对此进行了明确的区分。
- 输入为单文本时，输入序列为 \<cls> + 文本序列(标记化表示)$e_A$ + \<sep>。
- 输入为文本对时，输入序列为 \<cls> + 第一个文本序列的标记$e_A$ + \<sep> + 第二个文本序列标记$e_B$ + \<sep>

In [3]:
def get_tokens_and_segments(tokens_a, tokens_b=None):
    """获取输入序列的词元及其片段索引"""
    tokens = ['<cls>'] + tokens_a + ['<sep>']
    # 0和1分别标记片段A和B
    segments = [0] * (len(tokens_a) + 2)
    if tokens_b is not None:
        tokens += tokens_b + ['<sep>']
        segments += [1] * (len(tokens_b) + 1)
    # 返回的是一个单词数组，及等大的标记数组
    return tokens, segments

['<cls>', 'A', 'A', 'A', '<sep>']

## Bert 数据

## Bert 训练

## QA